<h1 style = 'background-color:black;color:white;font-family:Ubuntu'><center>✨ Kaggle in Tweets :What has changed in these 11 years? ✨ </center> </h1>

<div style ="display: block;margin-left: auto;margin-right: auto;width: 50%;">
    <img src = "https://miro.medium.com/max/1024/1*bQYZ7fp_wcjWFwpBbcRS-Q.png"/>
 </div>

<h1 style = 'background-color:black;color:white;font-family:Ubuntu'><center> About the Dataset And This Notebook </center> </h1>



🌟 Kaggle has been a temple for Data Scientists , Deep Learning Researchers and aspirants like me. This notebook is aimed at drawing comparision between the presence of kaggle on twitter in the year 2010 and that in the current year 2021 ,via Tweet Analysis. Before you scroll through the notebook , let me tell you a bit about the Data quickly.

```
 1.The dataset contains tweets regarding "Kaggle" from verified twitter accounts. 
 2.It has a record of nearly 11 years , i.e, from 2010 to 2021.
 3.These tweets have been scraped using Twint.

```
🌟 In this notebook , I have done analysis with English as the base language , for now. Will include updated analysis results in a revised version.

🌟 All right , that's all you need to do. Let's start ! 

<h1 style = 'background-color:black;color:white;font-family:Ubuntu'><center>Index</center> </h1>

1. [Libraries and Methods](#sec1)
2. [Time to Clean:Preprocessing and Cleaning Data](#sec2)
3. [Year in Review : 2010](#sec3)
4. [Conclusion of Year 2010](#sec4)
5. [Year in Review : 2021](#sec5)
6. [Conclusion of Year 2021](#sec6)
7. [What has really changed?](#sec7)

<h1 id = 'sec1' style = 'background-color:black;color:white;font-family:Ubuntu'><center>Libraries and Methods</center> </h1>  

# **Installing Necessary Libraries and Importing Them**

In [ ]:
!pip install contractions
!pip install keyBERT
!pip install transformers
from keybert import KeyBERT
model = KeyBERT('distilbert-base-nli-mean-tokens')
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import re
import contractions
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
words=set(nltk.corpus.words.words())
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
from collections import Counter
from wordcloud import WordCloud
import warnings
warnings.filterwarnings("ignore")
output_notebook()

# **Methods**

In [ ]:
def check_null(data):
    for i in data.columns:
        print(i,":",(data[str(i)].isnull().sum()/data.shape[0])*100)

In [ ]:
def to_delete_cols(data):
    to_delete = []
    for i in data.columns:
        if((data[str(i)].isnull().sum()/data.shape[0])*100 > 95):
            to_delete.append(i)
    return to_delete

In [ ]:
def clean_txt(input_txt, pattern = "@[\w]*"):
    #removing hashtags,emojis,stopwords
    input_txt=re.sub(r'#[\w]*','',input_txt)
    input_txt=input_txt.encode("ascii","ignore")
    input_txt=input_txt.decode()
    
    ##removing @user
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
   
    #removing stopwords    
    input_txt = ' '.join([i for i in input_txt.split() if not i in words])
    #contractions
    input_txt=contractions.fix(input_txt)
    #removing punctuation,numbers and whitespace   
    res=re.sub(r'[^\w\s]', '', input_txt.lower())
    res=re.sub('\s+',' ',res)
    ##removing links
    res=re.sub(r'https[\w]*', '', res, flags=re.MULTILINE)
    #removing acronyms
    res=''.join(i for i in res if not i.isdigit())
    res=' '.join([i for i in res.split() if len(i)>2])
    lem = WordNetLemmatizer()
    res = lem.lemmatize(res)
    return res

In [ ]:
def get_keywords(x):
    list_keywords = []
    keywords = model.extract_keywords(x, keyphrase_ngram_range=(1, 1))
    for i in keywords:
        list_keywords.append(i[0])
    return list_keywords

In [ ]:
def get_keywords_ratings(df):
    keywords = []
    for _,row in df.iterrows():
        for i in row['Keywords']:
            keywords.append(i)
    keywords = list(set(keywords))
    keywords_percentage = dict.fromkeys(set(keywords),0)
    for _,row in df.iterrows():
        for i in row['Keywords']:
            keywords_percentage[i]+=1
    
    return pd.DataFrame.from_dict(keywords_percentage,orient = 'index',columns = ['Frequency']).reset_index().rename(columns = {'index':'Words'}).sort_values('Frequency',ascending = False).reset_index()

In [ ]:
def get_tokens(x):
    return x.split()

In [ ]:
def vectorize(data):
    tfidf_vectorizer = TfidfVectorizer()
    emb = tfidf_vectorizer.fit_transform(data)
    return emb,tfidf_vectorizer

In [ ]:
def get_keys(topic_matrix):
    '''
    returns an integer list of predicted topic 
    categories for a given topic matrix
    '''
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):
    '''
    returns a tuple of topic categories and their 
    accompanying magnitudes for a given list of keys
    '''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)



In [ ]:
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer,n_topics = 8):
    '''
    returns a list of n_topic strings, where each string contains the n most common 
    words in a predicted category, in order
    '''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [ ]:
def get_hashtags(data):
    hashtags = ""
    for _,row in data.iterrows():
        a=""
        for i in row['hashtags']:
            if(i == '[' or i == "'" or i == ']'):
                continue
            else:
                a+=i
        if(a.split(',') != ['']):
            hashtags+=a
            hashtags+=','
    return hashtags.split(',')

In [ ]:
def get_frequency_hashtags(data):
    hashtags = get_hashtags(data)
    freq_hashtags = {'Hashtag':[],'Frequency':[]}
    for i in set(hashtags):
        if (i !=''):
            freq_hashtags['Hashtag'].append(i)
            freq_hashtags['Frequency'].append(hashtags.count(i))
    return pd.DataFrame.from_dict(freq_hashtags).sort_values('Frequency',ascending = False)

In [ ]:
def get_sentiment(x):
    if(classifier(x)[0]['label'] == 'POSITIVE'):
        return 1
    return 0

In [ ]:
def wc(data,bgcolor,title):
    plt.figure(figsize = (15,15))
    wc = WordCloud(background_color = bgcolor, max_words = 1000,  max_font_size = 50)
    wc.generate(' '.join(data))
    plt.title(title , fontsize = 20)
    plt.imshow(wc)
    plt.axis('off')

In [ ]:
def get_mean_topic_vectors(keys, two_dim_vectors):
    '''
    returns a list of centroid vectors from each predicted topic category
    '''
    mean_topic_vectors = []
    for t in range(8):
        articles_in_that_topic = []
        for i in range(len(keys)):
            if keys[i] == t:
                articles_in_that_topic.append(two_dim_vectors[i])    
        
        articles_in_that_topic = np.vstack(articles_in_that_topic)
        mean_article_in_that_topic = np.mean(articles_in_that_topic, axis=0)
        mean_topic_vectors.append(mean_article_in_that_topic)
    return mean_topic_vectors

<h1 id = 'sec2' style = 'background-color:black;color:white;font-family:Ubuntu'><center>Time to Clean : Preprocessing and Preparing Data For Analysis</center> </h1>

# **Loading Files**

In [ ]:
data = pd.read_csv('../input/kaggle-tweets-2010-2021/KaggleTweets2010.csv')
for i in range(2011,2019):
    print(i)
    new_data = pd.read_csv('../input/kaggle-tweets-2010-2021/KaggleTweets{year}.csv'.format(year = str(i)))
    data = data.append(new_data)

In [ ]:
data = data.append(pd.read_csv('../input/kaggle-tweets-2010-2021/KaggleTweets2019Part1.csv'))
data = data.append(pd.read_csv('../input/kaggle-tweets-2010-2021/KaggleTweets2019Part2.csv'))
data = data.append(pd.read_csv('../input/kaggle-tweets-2010-2021/KaggleTweets2020Part1.csv'))
data = data.append(pd.read_csv('../input/kaggle-tweets-2010-2021/KaggleTweets2020Part2.csv'))
data = data.append(pd.read_csv('../input/kaggle-tweets-2010-2021/KaggleTweets2021.csv'))
data.tail()

In [ ]:
data.head()

# **Preprocessing and Cleaning The Data**

In [ ]:
check_null(data)

We will be deleting these values.

In [ ]:
data.drop(columns = to_delete_cols(data) , axis = 1 , inplace = True)
data.head()

In [ ]:
check_null(data)

In [ ]:
data.info()

In [ ]:
data.drop(columns = ['timezone','photos','video','reply_to','username','thumbnail','name','date','time'],axis = 1,inplace = True)
data['created_at'] = pd.to_datetime(data['created_at'])
data.head()

In [ ]:
def clean_txt(input_txt, pattern = "@[\w]*"):
    #removing hashtags,emojis,stopwords
    input_txt=re.sub(r'#[\w]*','',input_txt)
    input_txt=input_txt.encode("ascii","ignore")
    input_txt=input_txt.decode()
    
    ##removing @user
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
   
    #removing stopwords    
    input_txt = ' '.join([i for i in input_txt.split() if not i in words])
    #contractions
    input_txt=contractions.fix(input_txt)
    #removing punctuation,numbers and whitespace   
    res=re.sub(r'[^\w\s]', '', input_txt.lower())
    res=re.sub('\s+',' ',res)
    ##removing links
    res=re.sub(r'https[\w]*', '', res, flags=re.MULTILINE)
    #removing acronyms
    res=''.join(i for i in res if not i.isdigit())
    res=' '.join([i for i in res.split() if len(i)>2])
    lem = WordNetLemmatizer()
    res = lem.lemmatize(res)
    return res

# **Decoding Languages**

I looked up twitter's language codes to decode the given representations. I was unable to find certain codes and have left them as such.

In [ ]:
language = {'en' : 'English' , 'ja' : 'Japanese' , 'ht' : 'ht' , 'es' : 'Spanish' , 'zh' : 'Chinese' , 'it' : 'Italian' , 'und' : 'und' , 'fr' : 'French' , 'de' : 'German'
           , 'ru' : 'Russian', 'pt' : 'Portuguese' , 'ko' : 'Korean' , 'nl' : 'Dutch' , 'ro' : 'Romanian' , 'uk' : 'Ukrainian' , 'in' : 'in' , 'cy' : 'cy' , 'tl' : 'tl' , 'no' : 'Norwegian', 'hu':'hu', 'sl':'sl',
       'da' : 'Danish', 'el' : 'Greek', 'sv':'Swedish', 'lt': 'lt', 'et':'et', 'tr' : 'Turkish', 'lv' : 'lv', 'cs' : 'Czech', 'pl' : 'Polish', 'fi' : 'Finnish', 'ar' : 'Arabic',
       'is':'is', 'th' : 'Thai', 'fa' : 'Persian', 'bg' : 'bg',  'hi' : 'Hindi', 'iw' : 'iw', 'vi' : 'Vietnamese', 'eu' : 'eu', 'lo' : 'lo', 'ta':'ta', 'te':'te',
       'ur': 'Urdu', 'bn' : 'Bengali', 'ne' : 'ne', 'mr' : 'mr'}
data['language'] = data['language'].map(language)

In [ ]:
data['language'] = data['language'].replace(np.nan , 'LCode Not Found')

In [ ]:
data['language'].unique()

<h1 id = 'sec3' style = 'background-color:black;color:white;font-family:Ubuntu'><center>Year in Review : 2010</center> </h1>

In [ ]:
data_2010 = data[data['created_at'].dt.year == 2010]
data_2010.head()

# **Most Frequently Used Languages In Tweets 2010**

In [ ]:
data_2010['language'].value_counts().index[:5]

In [ ]:
language_series = pd.DataFrame({ 'Languages': data_2010['language'].value_counts().index , 'Total Tweets' : data_2010['language'].value_counts().tolist()})
language_series

In [ ]:
fig,ax = plt.subplots(ncols = 2, figsize = (20,4) , dpi = 100)
plt.tight_layout()
colors = ['#3d5a80', '#98c1d9' , '#e0fbfc' ,'#ee6c4d' , '#293241' , '#ffd92f','#e5c494']
language_series['Total Tweets'][:5].plot(kind = 'pie', labels = [' ',' ',' ',' ',' '], colors = colors[:5] , ax = ax[0])
sns.barplot(y = language_series['Languages'][1:5] , x = language_series['Total Tweets'][1:5] ,ax = ax[1], palette = colors[:4])
ax[0].set_ylabel('')
ax[0].legend(labels = language_series['Languages'])
ax[0].set_title('Most Frequently Used Languages' ,  fontsize = 10)
ax[1].set_title('Subtracting English From the Equation',fontsize = 10)

# **User Ids Of Top 20 Tweet Accounts 2010**

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(y = [str(i) for i in data_2010['user_id'].unique()][:20] , x = data_2010['user_id'].value_counts().values[:20] , color = '#293241')
plt.ylabel('User Ids' , fontsize = 15)
plt.xlabel('Number of Tweets', fontsize = 15)
plt.title('Top 20 Tweeters' , fontsize = 20)

# **Monthly Count of Total Tweets 2010**

In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize = (13,10))
colors = ['#ee6c4d' for i in range(10)]
colors[3] = '#293241'
colors[5] = '#293241'
colors[7] = '#293241'
colors[8] = '#293241'
sns.countplot(data_2010['created_at'].dt.month , palette = colors)
plt.xlabel('Months',fontsize = 13)
plt.ylabel('')
plt.title('Count of Tweets Monthly',fontsize = 18)

The peak months were clearly Novemeber, October, August and June.

# **Total Tweets Segregated Hourwise 2010**

In [ ]:
plt.figure(figsize = (15,10))
colors = ['#ee6c4d' for i in range(24)]
colors[23] = '#293241'
sns.countplot(data_2010['created_at'].dt.hour , palette = colors)
plt.xlabel('Hours',fontsize = 15)
plt.ylabel('')
plt.title('Count of Tweets Hourly',fontsize = 20)

Clearly, I am not alone in scrolling through social media at the wrong time. From this we can see that for most of the office hours that is 9am-5pm, the number of tweets are less as compared to the late hours at night. Especially 11 pm. We'll see if this trend is continued for the next years.

# **Top 30 Keywords From Tweets of 2010**

We can see that there are extremely few tweets in other languages , so just for now, in the rest of the notebook I  have provided the analysis on the basis of the tweets in english.

In [ ]:
data_2010 = data_2010[data_2010['language'] == 'English']
data_2010['Clean Tweet'] = data_2010['tweet'].apply(clean_txt)
data_2010['Keywords'] = data_2010['Clean Tweet'].apply(get_keywords)
data_2010['Sentiment-Tweet'] = data_2010['tweet'].apply(get_sentiment)
data_2010.head()

In [ ]:
keywords_2010 = get_keywords_ratings(data_2010).drop(columns = ['index'], axis=0)[:30]
plt.figure(figsize = (20,20))
sns.barplot(y = keywords_2010['Words'][1:] , x = keywords_2010['Frequency'][1:] , color = '#293241')
plt.tick_params(axis = 'y',labelsize = 12)
plt.xlabel('Frequency', fontsize = 15)
plt.ylabel('Keywords' , fontsize = 15)
plt.title('Top 30 Keywords from Tweets of 2010' , fontsize = 20)

# **Topic Modelling using LDA 2010**

In [ ]:
list_corpus,tfidf = vectorize(data_2010["Clean Tweet"].tolist())

colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5" ])
colormap = colormap[:8]

In [ ]:
n_topics = 8
lda_model = LatentDirichletAllocation(n_components=8, learning_method='online', 
                                          random_state=0, verbose=0)
lda_topic_matrix = lda_model.fit_transform(list_corpus)

lda_keys = get_keys(lda_topic_matrix)
lda_categories, lda_counts = keys_to_counts(lda_keys)

top_n_words_lda = get_top_n_words(10, lda_keys,list_corpus, tfidf)

for i in range(len(top_n_words_lda)):
    print("Topic {}: ".format(i+1), top_n_words_lda[i])

# **Top 8 Topics Found Using LDA**

In [ ]:
top_3_words = get_top_n_words(3, lda_keys, list_corpus, tfidf)
labels = ['Topic {}: \n'.format(i) + top_3_words[i] for i in lda_categories]

fig, ax = plt.subplots(figsize=(16,8))
ax.bar(lda_categories, lda_counts , color = '#293241')
ax.tick_params(axis = 'x' , labelsize = 12)
ax.set_xticks(lda_categories)
ax.set_xticklabels(labels)
ax.set_title('LDA topic counts' , fontsize = 20)
ax.set_ylabel('Number of headlines', fontsize = 15)
plt.xticks(rotation = 45)



# **TSNE Representation Of Top 8 Topics Using LDA 2010**

In [ ]:
tsne_lda_model = TSNE(n_components=2, perplexity=50, learning_rate=100, 
                        n_iter=2000, verbose=1, random_state=0, angle=0.75)
tsne_lda_vectors = tsne_lda_model.fit_transform(lda_topic_matrix)

In [ ]:
top_3_words_lda = get_top_n_words(3, lda_keys, list_corpus, tfidf)
lda_mean_topic_vectors = get_mean_topic_vectors(lda_keys, tsne_lda_vectors)

plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), plot_width=700, plot_height=700)
plot.scatter(x=tsne_lda_vectors[:,0], y=tsne_lda_vectors[:,1], color=colormap[lda_keys])

for t in range(n_topics):
    label = Label(x=lda_mean_topic_vectors[t][0], y=lda_mean_topic_vectors[t][1], 
                  text=top_3_words_lda[t], text_color=colormap[t])
    plot.add_layout(label)

show(plot)

# **Top Hashtags of The Year 2010**

In [ ]:
frequency_hashtags = get_frequency_hashtags(data_2010)[:20]
plt.figure(figsize = (20,20))
sns.barplot(y = frequency_hashtags['Hashtag'] , x = frequency_hashtags['Frequency'] , color = '#293241')
plt.tick_params(axis = 'y',labelsize = 12)
plt.ylabel('Hashtags' , fontsize = 15)
plt.xlabel('Frequency', fontsize = 15)
plt.title('Top 20 Hashtags from Tweets of 2010' , fontsize = 18)

# **Sentiment Analysis Of Tweets 2010**

In [ ]:
plt.figure(figsize = (10,10))
colors_senti = ['#ee6c4d' , '#3d5a80']
sns.countplot(data_2010['Sentiment-Tweet'], palette = colors_senti)
plt.xlabel('Sentiment',fontsize = 10)
plt.ylabel('')
plt.title('Assigned Sentiment Distribution',fontsize = 15)

In [ ]:
positive_tweets = data_2010[data_2010['Sentiment-Tweet'] == 1]['Clean Tweet'].tolist()
positive_tweets = list(set(positive_tweets))
negative_tweets = data_2010[data_2010['Sentiment-Tweet'] == 0]['Clean Tweet'].tolist()
negative_tweets = list(set(negative_tweets))
common_words = list(set(positive_tweets)&set(negative_tweets))
positive_tweets = list(set(positive_tweets).difference(set(common_words)))
negative_tweets = list(set(negative_tweets).difference(set(common_words)))

# **Word Cloud For Positive Tweets 2010**

In [ ]:
wc(positive_tweets,'black','Positive Ones')

# **Word Clouds for Negative Tweets 2010**

In [ ]:
wc(negative_tweets,'black','Negative Ones')

<h1 id = 'sec4' style = 'background-color:black;color:white;font-family:Ubuntu'><center> Conclusion of 2010 </center> </h1>



1. **Most Frequently Used Languages** 
    *  English is the most frequently used language as it appeared in more than 96% of Data, followed by an unknown language code 'und' which had only 5 tweets to its name.

2. **Top 20 Tweeters** 
    * I used User Ids instead of names for the sake of privacy. From the Graph , we can clearly see which user id was the most active on tweeting about Kaggle.

3. **Peak Months** 
    * The most tweets were recorded in the month of Novemeber , followed by October , August and June. Below I have discussed possible reasons for trends in these particular months:
        * November : The winners of the INFORMS Data Mining Contest were honoured in Austin, Texas in November in a huge event.
        * October : Results for INFORMS Data Mining Contest and HIV Progression Prediction Competition were announced. 
        * June : A Dataset was relased by JPMorgan and other leading Investment Banks on Kaggle , challenging Data Scientists to predict the winner for the 2010 Fifa World Cup.It was between 3th June - 6th June. The Fifa World Cup started on the 11th of June. 
        * I tried looking for explanation for the other months but couldn't find any , so far. Will Update once I find something.

4. **Most Frequent Hours for incoming Tweets**
    * The Twitter Data clearly shows that even a decade back , people were more active on twitter during Night Time. 
    *  The late night hours between 10 pm to 2 am have more high frequencies of tweets than any other period observed. 
    *  The period b/w 12pm to 8 pm registered the least amount of tweets. This could be because of the office hours.
    
5. **Top 30 Keywords** 
    * This list is topped by the word competition and is followed by words like data mining , bioinformatics , hiv and goldbloom.
    * The word data mining is in the top because of the INFORMS Data Mining Event , which I covered in the Peak Months.
    * Anthony Goldbloom is the CFO and CEO of Kaggle and Kaggle started in the year 2010.
    * Bioinformatics and HIV Progression were competitions which were launched by Kaggle.

6. **Topic Modelling**
    *  Through Topic Modelling few important topics of discussions were identified most of which were talks about competitions. They are as follows:
          * Melbourne : A competition for predicting the outcome of grants for University of Melbourne was launched.
          * HIV : I already discussed this.
          * Rest topics found were either a link to the Kaggle Website or a thanks statement or perhaps appreciation of any competition.     
          
7. **Top 30 Hashtags**
   * The top most hashtag was of course of the word kaggle . It was followed closely by other words such as chess , world cup , gov2au , kdnuggets , etc.
   * gov2au : Another Data Science Competition related to Travel Analysis was launched by the Government of Australia. People were tasked with predicting the Travel Time on Sydney's M4 freeway.

<h1 id='sec5' style = 'background-color:black;color:white;font-family:Ubuntu'><center> Year in Review : 2021</center> </h1>

In [ ]:
data_2021 = data[data['created_at'].dt.year == 2021]
data_2021.to_csv('data_2021.csv')
data_2021.head()

In [ ]:
data_2021.info()

# **Most Frequently Used Languages In Tweets 2021**

In [ ]:
language_series = pd.DataFrame({ 'Languages': data_2021['language'].value_counts().index , 'Total Tweets' : data_2021['language'].value_counts().tolist()})
language_series.head()

In [ ]:
fig,ax = plt.subplots(ncols = 3, figsize = (28,5) , dpi = 100)
colors = ['#780000', '#c1121f' , '#fdf0d5' ,'#003049' , '#669bbc']
language_series['Total Tweets'][:5].plot(kind = 'pie', labels = [' ',' ',' ',' ',' '], colors = colors , ax = ax[0])
sns.barplot(y = language_series['Languages'][:2] , x = language_series['Total Tweets'][:2] ,ax = ax[1], palette = [colors[1],colors[3]])
sns.barplot(y = language_series['Languages'][2:6] , x = language_series['Total Tweets'][2:6] ,ax = ax[2], palette = colors[:4])
ax[0].set_ylabel('')
ax[0].legend(labels = language_series['Languages'])
ax[0].set_title('Most Frequently Used Languages' ,  fontsize = 15)
ax[1].set_title('Comparision between Japanese and English' , fontsize = 15)
ax[2].set_title('Subtracting Japanese and English From the Equation',fontsize = 15)

# **Top 20 Tweeters 2021**

In [ ]:
plt.figure(figsize=(20,20))
sns.barplot(y = [str(i) for i in data_2021['user_id'].unique()][:20] , x = data_2021['user_id'].value_counts().values[:20] , color = colors[1])
plt.ylabel('User Ids',fontsize = 15)
plt.xlabel('Number of Tweets',fontsize = 15)
plt.title('Top 20 Tweeters' , fontsize = 20)

# **Monthly Count of Total Tweets 2021**

In [ ]:
plt.figure(figsize = (8,8))
colors_months = [colors[1] for i in range(4)]
colors_months[0] = colors[3]
sns.countplot(data_2021['created_at'].dt.month , palette = colors_months)
plt.xlabel('Months',fontsize = 10)
plt.ylabel('')
plt.title('Count of Tweets Monthly',fontsize = 15)

Maximum tweets , so far , seem to have come from the month of January.

# **Total Tweets Segregated Hourwise 2021**

In [ ]:
fig,ax = plt.subplots(ncols = 2 , figsize = (20,10))
pale = [colors[1] for i in range(24)]
for i in range(8,17):
    pale[i] = colors[4]
pale[14] = colors[3]
pale[13] = colors[3]
sns.countplot(data_2021['created_at'].dt.hour , palette = pale , ax = ax[0])
sns.distplot(data_2021['created_at'].dt.hour , ax = ax[1])
ax[0].set_xlabel('Hours',fontsize = 10)
ax[0].set_ylabel('')
ax[0].set_title('Count of Tweets Hourly',fontsize = 15)
ax[1].set_title('Tweet Distribution Hourly' , fontsize = 15)
ax[1].set_xlabel('Hours' , fontsize = 10)

It looks like people have become extremely addicted to social media by now. The peak tweeting hours have however shifted b/w 8 am to 4 pm with 2 pm recording the most tweets in the first 4 months of this year.

# **Keywords of Tweets of Year 2021**

The majority of texts are in Japanese. However , I am yet to train the MBERT embeddings for the task of Sentiment Analysis. Will include the results , after having done that.

In [ ]:
data_2021 = data_2021[data_2021['language'] == 'English']
data_2021['Clean Tweet'] = data_2021['tweet'].apply(clean_txt)
data_2021['Keywords'] = data_2021['Clean Tweet'].apply(get_keywords)
data_2021['Sentiment-Tweet'] = data_2021['tweet'].apply(get_sentiment)
data_2021.head()

In [ ]:
keywords_2021 = get_keywords_ratings(data_2021).drop(columns = ['index'], axis=0)[:30]
plt.figure(figsize = (20,20))
sns.barplot(y = keywords_2021['Words'][1:] , x = keywords_2021['Frequency'][1:] , color = colors[1])
plt.tick_params(axis = 'y',labelsize = 12)
plt.title('Top 30 Keywords from Tweets of 2010' , fontsize = 15)


# **Topic Modelling 2021**

In [ ]:
list_corpus,tfidf = vectorize(data_2021["Clean Tweet"].tolist())

In [ ]:
n_topics = 8
lda_model = LatentDirichletAllocation(n_components=8, learning_method='online', 
                                          random_state=0, verbose=0)
lda_topic_matrix = lda_model.fit_transform(list_corpus)

lda_keys = get_keys(lda_topic_matrix)
lda_categories, lda_counts = keys_to_counts(lda_keys)

top_n_words_lda = get_top_n_words(10, lda_keys,list_corpus, tfidf)

for i in range(len(top_n_words_lda)):
    print("Topic {}: ".format(i+1), top_n_words_lda[i])

In [ ]:
top_3_words = get_top_n_words(3, lda_keys, list_corpus, tfidf)
labels = ['Topic {}: \n'.format(i) + top_3_words[i] for i in lda_categories]

fig, ax = plt.subplots(figsize=(16,8))
ax.bar(lda_categories, lda_counts , color = colors[1])
ax.set_xticks(lda_categories)
ax.set_xticklabels(labels)
ax.set_title('LDA topic counts')
ax.set_ylabel('Number of headlines')
plt.xticks(rotation = 45)

In [ ]:
tsne_lda_model = TSNE(n_components=2, perplexity=50, learning_rate=100, 
                        n_iter=2000, verbose=1, random_state=0, angle=0.75)
tsne_lda_vectors = tsne_lda_model.fit_transform(lda_topic_matrix)

In [ ]:
top_3_words_lda = get_top_n_words(3, lda_keys, list_corpus, tfidf)
lda_mean_topic_vectors = get_mean_topic_vectors(lda_keys, tsne_lda_vectors)

plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), plot_width=700, plot_height=700)
plot.scatter(x=tsne_lda_vectors[:,0], y=tsne_lda_vectors[:,1], color=colormap[lda_keys])

for t in range(n_topics):
    label = Label(x=lda_mean_topic_vectors[t][0], y=lda_mean_topic_vectors[t][1], 
                  text=top_3_words_lda[t], text_color=colormap[t])
    plot.add_layout(label)

show(plot)

# **Top 20 Hashtags from Tweets of 2021**

In [ ]:
frequency_hashtags = get_frequency_hashtags(data_2021)[:20]
plt.figure(figsize = (20,20))
sns.barplot(y = frequency_hashtags['Hashtag'] , x = frequency_hashtags['Frequency'] , color = colors[1])
plt.tick_params(axis = 'y',labelsize = 12)
plt.title('Top 20 Hashtags from Tweets of 2021' , fontsize = 15)

# **Sentiment Analysis of Tweets 2021**

In [ ]:
plt.figure(figsize = (10,10))
pale = [colors[1] , colors[4]]
sns.countplot(data_2021['Sentiment-Tweet'], palette = pale)
plt.xlabel('Sentiment',fontsize = 10)
plt.ylabel('')
plt.title('Assigned Sentiment Distribution',fontsize = 15)

In [ ]:
positive_tweets = data_2021[data_2021['Sentiment-Tweet'] == 1]['Clean Tweet'].tolist()
positive_tweets = list(set(positive_tweets))
negative_tweets = data_2021[data_2021['Sentiment-Tweet'] == 0]['Clean Tweet'].tolist()
negative_tweets = list(set(negative_tweets))
common_words = list(set(positive_tweets)&set(negative_tweets))
positive_tweets = list(set(positive_tweets).difference(set(common_words)))
negative_tweets = list(set(negative_tweets).difference(set(common_words)))

# **Positive Word Cloud 2021**

In [ ]:
wc(positive_tweets,'black','Positive Ones')

# **Negative Word Cloud 2021**

In [ ]:
wc(negative_tweets,'black','Negative Ones')

<h1 id = 'sec6' style = 'background-color:black;color:white;font-family:Ubuntu'><center> Conclusion of 2021</center> </h1>


1. **Most Frequently Used Languages** 
    * Japanese and English dominate the Languages Region. 
    * This could imply that Kaggle has a huge fan base in Japan.

2. **Top 20 Tweeters** 
    * I used User Ids instead of names for the sake of privacy. From the Graph , we can clearly see which user id was the most active on tweeting about Kaggle.

3. **Peak Months** 
    * The peak month so far has been January.
4. **Most Frequent Hours for incoming Tweets**
    * Well, the number of incoming Tweets regarding Kaggle have increased as compared to the numbers in 2010. 
    * Another thing which has changed is that most number of tweets are found in the 8am to 4pm period.
    
5. **Top 30 Keywords** 
    * This list is topped by the words learning, python , competitions and grandmaster.
    * It certainly shows a vast difference as compared to the starting days of Kaggle in 2010. The only thing in common with that year and 2021 are the tweets regarding competition.    

6. **Topic Modelling**
    *  Through Topic Modelling few important topics of discussions were identified most of which were talks about competitions but no particular competitions came up so far.
7. **Top 20 Hashtags**
   * The top most hashtag was of course of the word kaggle . It was followed closely by other words such as data science, machine learning and 66daysofData.
   * There are many challenges which can be seen in the list. This makes one feel that the tweets related to Kaggle in 2021 are all about learning.
   

<h1 id = 'sec7' style = 'background-color:black;color:white;font-family:Ubuntu'><center> So, what really has changed?</center> </h1>

> In 2010 , when kaggle had just started , most of the people who tweeted about it were using english as a medium to share their experience. The incoming tweets were observed to increase when a new competition was launched and also , during late night hours. Most people used to tweet about an ongoing competition or perhaps something related to statistics as the word 'stats' came across a lot. If ,an overall image were to be formed for this year, it would probably have the names of the famous competitions in this year and a lot of congratulatory messages(which can be seen in the word cloud of 2010). This year can be called the year of competitions according to me.

> In 2021, the Japanese language was the language which was used more than often to share one's experience w.r.t kaggle. It was followed by English very closely. One cannot draw a conclusion or hypothesize any theory on the increase of tweets in a particular month as the months so far have been mostly neck to neck. The total number of incoming tweets increased by a huge margin and most of them seem to be coming in the time period of 8am to 4pm. Most of the people seem to be talking about learning , competitions and perhaps congratulating the newly annointed Grandmasters. The hastags also confirm this story as they seem to be dominated by challenges such as 100daysofml , which help people learn new things in the field of ML. Also, the topic of discussions have shifted from statistics to machine learning and AI.

 **So,one can say that 2010 was all really about the competitions held on kaggle and statistics, whereas 2021 has become all about learning new concepts,the titles of masters and grandmasters , competitions, Machine Learning and Artificial Intelligence.**

<h1 style = 'background-color:black;color:white;font-family:Ubuntu'><center> References </center> </h1>

[Topic Modelling ](https://www.kaggle.com/rcushen/topic-modelling-with-lsa-and-lda)

<h1 style = 'background-color:black;color:white;font-family:Ubuntu'><center> That's all folks ! </center> </h1>

Thanks for checking out my notebook. Liked my work , leave an upvote ! Found something and want to discuss it , leave a comment and i'll get back to you ASAP 😊
